# Quantitative Research Case Study

Looking at a binomial model, we consider discrete time periods with two optional moves at each period. The move up and down is also equal. Given this information, we know that it is an arbitrage-free and complete market. Hence, we can use risk-neutral probability measures to compute option prices. We also notice that since the percentage of which the value of the underlying asset goes up/down at each time period is equal, then the theoretical risk-neutral probability will also be equal (i.e. 0.5).

In [2]:
import math

def combos(n, i):
    return math.factorial(n) / (math.factorial(n-i)*math.factorial(i))

def risk_neutral_probabilities(N):    
    p = 0.5
    q = list()
    for k in reversed(range(N+1)):
        q.append(combos(N, k)*p**k *(1-p)**(N-k))
        return q

def eurpoean_option_price(v, k, N):
    if N < 1: raise ValueError("N is incurrect.")
    if type(v) != float or v >= 1 or v <= 0: raise ValueError("v is incurrect.")
    if (type(k) != float and type(k) != int) or type(N) != int: raise TypeError("Strike or number of periods is from wrong data type.")
    termination_price = list()
    current = (1+v)**N
    last = (1-v)**N
    while current > k:
        termination_price.append(current - k)
        if current == last: break
        current = (current / (1+v)) * (1-v)
    q = risk_neutral_probabilities(N)
    return sum([termination_price[i] * q[i] for i in range(len(termination_price))])

# Please insert the input values below
v = 0.05
strike = 1
N = 2
print("The European Call option's price is: ", round(eurpoean_option_price(v, strike, N), 5))

The European Call option's price is:  0.02563


We can reliably build our next function based on the current one. Our approach is minimizing the difference between the call option's input value and the computed call option's value by our function. Our function then becomes a function of v, which we will change\tune correspondingly, to minimize the difference between the function's value and the given option's value. We don't even need to validate the input values, since this is already done by our previous function, which is called in the following implementation.

In [3]:
def find_v(V, k, N):
    v = 0.5
    low = 0.0
    high = 1.0
    runs = 0
    while abs(V - eurpoean_option_price(v, k, N)) > 0.000001:
        if runs > 10000: break
        if eurpoean_option_price(v, k, N) > V:
            high = v
            v = (v + low) / 2
        else:
            low = V
            v = (v + high) / 2
        runs += 1
    return v

# Please insert the input values below
V = 0.02563
strike = 1
N = 2
print("The corresponding value of v is: ", round(find_v(V, strike, N), 4))

The corresponding value of v is:  0.05


This part we are going to implement in a different way, which calculates each relevant node, until we get the value of the first node, which is the option's value.

In [4]:
import numpy as np

def american_call_pricing(v, k, N):
    u, d = 1+v, 1-v
    payoff = np.zeros(N+1)
    for j in range(N+1):
        payoff[j] = max(u**j * d**(N-j) - 1, 0)
    
    for i in np.arange(N-1, -1, -1):
        for j in range(0, i+1):
            S = u**j * d**(N-j)
            payoff[j] = max((payoff[j] + payoff[j+1])/2, S - k)
            
    return payoff[0]

            
# Please insert the input values below
v = 0.05
strike = 1
N = 2
print("The European Call option's price is: ", round(american_call_pricing(v, strike, N), 5))

The European Call option's price is:  0.02563


Here, given v, we will calculate the expected value of the maximum value of the underlying asset between the periods 0 and N. Mathematically speaking, we will compute:

$$ E[max_{0<j<N}S_j | v] $$

In [5]:
def expected_maximum(v, N):
    max_S = (1+v)**N
    return np.mean(max_S)

print(expected_maximum(0.08, 2))

1.1664
